# Osa 5: Indeksit
Tässä osassa luodaan erilaisia indeksejä ja tarkastellaan niiden vaikutusta.

In [ ]:
# Lataa SQL-lisäosa
%load_ext sql
# Luo tietokantayhteys tietokantaan postgres
from ipygis import get_connection_url
c_url = get_connection_url()
%sql $c_url

Luetaan taulusta kuinka monella rivillä ajoneuvoluokka on M1.

In [ ]:
import time
t = time.time()
res = %sql SELECT count(*) FROM tieliikenne WHERE ajoneuvoluokka = 'M1'
print(f"Kyselyn lukemiseen meni {time.time() -t:5.2} s")
res

## Indeksien luominen
Luodaan yksinkertainen B-TREE -indeksi sarakkeelle `ajoneuvoluokka`:

In [ ]:
%sql CREATE INDEX ajoneuvoluokka_idx ON tieliikenne(ajoneuvoluokka)

Tarkista nyt uudestaaan, kuinka kauan suoritus kestää.

In [ ]:
t = time.time()
res = %sql SELECT count(*) FROM tieliikenne WHERE ajoneuvoluokka = 'M1'
print(f"Kyselyn lukemiseen meni {time.time() -t:5.2} s")
res

## Indeksien koko
Indeksit kasvattavat tietokannan kokoa. Aja seuraava solu nähdäksesi kaikkien`tieliikenne`-taulun indeksien koot:

In [ ]:
%sql SELECT pg_size_pretty(pg_indexes_size('tieliikenne'));

## Indeksien poistaminen
Indeksin voi poistaa syntaksilla: 
```sql
DROP INDEX <nimi>
``` 

In [ ]:
%sql DROP INDEX ajoneuvoluokka_idx

## **Harjoitus 5.1**: Spatiaalinen indeksi
Indeksit ovat erityisen tärkeitä paikkatiedon parissa työskennellessä. Yritä seuraavaa kyselyä aluksi aineistolla, jolla ei ole indeksejä.

```sql
WITH helsinki AS (SELECT geom, nimi FROM kunnat WHERE nimi = 'Helsinki') 

SELECT count(*) FROM digiroad_uusimaa a 
JOIN helsinki b ON ST_Intersects(b.geom, a.geom);
```

Kysely on yhdellä rivillä harjoitusympäristön luonteen vuoksi, jotta ajan pystyy mittaamaan. 

In [ ]:
t = time.time()
%sql DROP INDEX IF EXISTS kunnat_geom_idx
%sql DROP INDEX IF EXISTS digiroad_geom_idx
res = %sql WITH helsinki AS (SELECT geom, nimi FROM kunnat WHERE nimi = 'Helsinki') SELECT count(*) FROM digiroad_uusimaa a JOIN helsinki b ON ST_Intersects(b.geom, a.geom)
print(f"Kyselyn suorittamiseen meni {time.time() -t:5.2} s")
res

Luo nyt spatiaalinen indeksi `kunnat_geom_idx` taululle kunnat syntaksilla

```sql
CREATE INDEX nimi ON taulu USING GIST ("geom")
```

In [ ]:
%sql 

In [ ]:
%sql CREATE INDEX kunnat_geom_idx ON kunnat USING GIST ("geom")

Mittaa sitten uudelleen aika kyselyn suorittamiseen, nopeutuiko suoritus?

In [ ]:
t = time.time()
%sql WITH helsinki AS (SELECT geom, nimi FROM kunnat WHERE nimi = 'Helsinki') SELECT count(*) FROM digiroad_uusimaa a JOIN helsinki b ON ST_Intersects(b.geom, a.geom)
print(f"Kyselyn suorittamiseen meni {time.time() -t:5.2} s")

Luo vielä spatiaalinen indeksi `digiroad_geom_idx` taululle `digiroad_uusimaa`

In [ ]:
%sql 

In [ ]:
%sql CREATE INDEX digiroad_geom_idx ON digiroad_uusimaa USING GIST ("geom")

Mittaa uudelleen aika

In [ ]:
t = time.time()
res = %sql WITH helsinki AS (SELECT geom, nimi FROM kunnat WHERE nimi = 'Helsinki') SELECT count(*) FROM digiroad_uusimaa a JOIN helsinki b ON ST_Intersects(b.geom, a.geom)
print(f"Kyselyn suorittamiseen meni {time.time() -t:5.2} s")

Visualisoidaan helsingin teitä. Kyselyn suorittamisessa menee hetki

In [ ]:
from ipygis import get_map
res = %sql WITH helsinki AS (SELECT geom, nimi FROM kunnat WHERE nimi = 'Helsinki') SELECT a.id, arvo, muokkauspv, a.geom FROM digiroad_uusimaa a JOIN helsinki b ON ST_Intersects(b.geom, a.geom)
map = get_map(res)
map

<a rel="license" href="http://creativecommons.org/licenses/by-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nd/4.0/88x31.png" />